In [1]:
include("/Users/luigibevilacqua/Desktop/AlphaShapes/src/AlphaStructures.jl")

Main.AlphaStructures

In [2]:
using LinearAlgebraicRepresentation, ViewerGL
using BenchmarkTools
using Distributed

In [3]:

Lar = LinearAlgebraicRepresentation
GL = ViewerGL
using TimerOutputs

#filename = "./OBJ/lowpolytree.obj";
filename = "/Users/luigibevilacqua/Desktop/AlphaShapes/examples/examples3D/OBJ/teapot.obj";
W, EVs, FVs = Lar.obj2lar(filename);
WW = [[i] for i = 1:size(W, 2)];
V, VV = Lar.apply(Lar.r(pi / 2, 0, 0), (W, WW)); #object rotated

points = convert(Lar.Points, V')
GL.VIEW([
    GL.GLPoints(points)
    GL.GLAxis(GL.Point3d(-1, -1, -1), GL.Point3d(1, 1, 1))
]);


filtration = AlphaStructures.alphaFilter(V);
VV, EV, FV, TV = AlphaStructures.alphaSimplex(V, filtration, 3.7)

GL.VIEW([
    GL.GLGrid(V, EV, GL.COLORS[1], 0.6) # White
    GL.GLGrid(V, FV, GL.COLORS[2], 0.3) # Red
    GL.GLGrid(V, TV, GL.COLORS[3], 0.3) # Green
]);

filter_key = sort(unique(values(filtration)))

granular = 10

reduced_filter =
    filter_key[sort(abs.(rand(Int, granular) .% length(filter_key)))]
reduced_filter = [reduced_filter; max(filter_key...)]


for α in reduced_filter
    #@code_warntype AlphaStructures.alphaSimplex(V, filtration, α)
    @show α
    VVV, EEV, FFV, TTV = AlphaStructures.alphaSimplex(V, filtration, α)
    """
    GL.VIEW(GL.GLExplode(
        V,
        [[[t] for t in TTV]; [[f] for f in FFV]; [[e] for e in EEV]],
        1.0,
        1.0,
        1.0,# Explode Ratio
        99,
        1,# Colors
    ))
    """
end

print_timer(AlphaStructures.to);


α = 1.745505896233702
α = 1.8233313214947122
α = 2.7952416021392783
α = 3.753104742173603
α = 4.4866498331366005
α = 4.591572653778346
α = 5.4066433534804785
α = 5.793201012281117
α = 7.120716815493177
α = 8.353590420251662
α = Inf
 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:             344s / 47.4%           13.1GiB / 73.4%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 alphaFilter                1     162s  99.3%    162s   9.45GiB  98.2%  9.45GiB
   delaunayTriangul...      1     151s  92.7%    151s   8.79GiB  91.3%  8.79GiB
     delaunayWall           1     151s  92.7%    151s   8.79GiB  91.3%  8.79GiB
       recursiveDel...      2     118s  72.5%  

                             518   63.5ms  0.04%   123μs   3.15MiB  0.03%  6.23KiB
                          531    143ms  0.09%   269μs   7.29MiB  0.07%  14.1KiB
                         2.86k   18.8ms  0.01%  6.57μs   1.29MiB  0.01%     474B
                         2.86k   1.74ms  0.00%   609ns    536KiB  0.01%     192B
                          515   29.6ms  0.02%  57.4μs   1.97MiB  0.02%  3.92KiB
                          515   2.35ms  0.00%  4.56μs    499KiB  0.00%     992B
                           45    480μs  0.00%  10.7μs   40.5KiB  0.00%     922B
                           16    188μs  0.00%  11.8μs   45.7KiB  0.00%  2.85KiB
                           16   1.93ms  0.00%   121μs    870KiB  0.01%  54.3KiB
                           16    803μs  0.00%  50.2μs    618KiB  0.01%  38.6KiB
                          605    14.9s  9.16%  24.7ms   1.00GiB  10.4%  1.69MiB
                          535    14.5s  8.88%  27.0ms    919MiB  9.33%  1.72MiB
                        51.0k    6.


signal (15): Terminated: 15
in expression starting at /Users/luigibevilacqua/.julia/packages/IJulia/IDNmS/src/kernel.jl:53
